In [1]:
import feather
import pandas as pd
from tqdm import tqdm

In [2]:
from outliers import remove_outliers

In [2]:
!ls data/

df_pays_agg.feather	 df_user_pay.feather	 shop_info.txt
df_pays_fixed.feather	 df_user_view.feather	 user_pay.txt
df_pays_na.feather	 df_views_agg.feather	 user_view.txt
df_pays_na_test.feather  prediction_example.csv
df_shop_info.feather	 shop1707.txt


In [4]:
df_shop_info = feather.read_dataframe('data/df_shop_info.feather')

In [6]:
from translate import translator

In [16]:
cities = sorted(set(df_shop_info.city_name))
result = {c: translator('zh-CN', 'en', c) for c in cities}
cities = {k: v[0][0][0] for k, v in result.items()}

In [80]:
import pickle
with open('tmp/cities.bin', 'wb') as f:
    pickle.dump(cities, f)

In [22]:
df_shop_info.city_name = df_shop_info.city_name.apply(cities.get)

In [44]:
from sklearn.preprocessing import LabelEncoder

In [47]:
df_shop_info['city_id'] = LabelEncoder().fit_transform(df_shop_info.city_name)

In [24]:
city_cnt = df_shop_info.city_name.value_counts()
df_shop_info['city_cnt'] = city_cnt[df_shop_info.city_name].reset_index(drop=1)
loc_cnt = df_shop_info.location_id.value_counts()
df_shop_info['loc_cnt'] = loc_cnt[df_shop_info.location_id].reset_index(drop=1)

In [50]:
df_shop_info['cat1_id'] = LabelEncoder().fit_transform(df_shop_info.category_1)
cat1_cnt = df_shop_info.cat1_id.value_counts()
df_shop_info['cat1_cnt'] = cat1_cnt[df_shop_info.cat1_id].reset_index(drop=1)

In [54]:
df_shop_info['cat2_id'] = LabelEncoder().fit_transform(df_shop_info.category_2)
cat2_cnt = df_shop_info.cat2_id.value_counts()
df_shop_info['cat2_cnt'] = cat2_cnt[df_shop_info.cat2_id].reset_index(drop=1)

In [56]:
df_shop_info['cat3_id'] = LabelEncoder().fit_transform(df_shop_info.category_3)
cat3_cnt = df_shop_info.cat3_id.value_counts()
df_shop_info['cat3_cnt'] = cat3_cnt[df_shop_info.cat3_id].reset_index(drop=1)

In [62]:
df_shop_info = df_shop_info.drop(['shop_id', 'city_name', 'category_1', 'category_2', 'category_3'], axis=1)

In [63]:
df_pays = feather.read_dataframe('data/df_pays_na_test.feather')

In [74]:
shops = df_pays.shop_id.unique()
shops = sorted(shops)

dfs = []

for i in tqdm(shops):
    df_shop = df_pays[df_pays.shop_id == i].reset_index(drop=1)
    df_info = df_shop_info.iloc[df_shop.shop_id - 1].reset_index(drop=1)
    df_shop = pd.concat([df_shop, df_info], axis=1)
    dfs.append(df_shop)

100%|██████████| 2000/2000 [00:07<00:00, 265.51it/s]


In [75]:
truncated_dfs = []

for df in dfs:
    df = df.iloc[7 * 2 * 3:]
    truncated_dfs.append(df)

In [76]:
df_features = pd.concat(truncated_dfs).reset_index(drop=1)

In [78]:
feather.write_dataframe(df_features, 'features/store_simple.feather')